In [ ]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings, BytePairEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from flair.datasets import ClassificationCorpus
from flair.data import Corpus
from hyperopt import hp
from flair.hyperparameter.param_selection import SearchSpace, Parameter

data_dir = "/depot/training/flair/data_sep2019/"
corpus: Corpus = ClassificationCorpus(data_dir, test_file='test.csv', dev_file='dev.csv',
                                      train_file='train.csv')

label_dict = corpus.make_label_dictionary()
word_embeddings = [FlairEmbeddings('multi-forward'), FlairEmbeddings('multi-backward'), BytePairEmbeddings('multi')]
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True,
                                            reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict,
                            multi_label=False)  # , multi_label_threshold=0.5)
trainer = ModelTrainer(classifier, corpus)

trainer.train(data_dir,
              learning_rate=0.2,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=3,
              max_epochs=20, embeddings_storage_mode='gpu')